In [ ]:
!pip install spacy -q
!python -m spacy download pl_core_news_sm
!pip install rapidfuzz -q

## 1. Użycie wbudowanego modelu spaCy

Model pl_core_news_sm w **spaCy** jest już wytrenowany do rozpoznawania lokalizacji (np. miast, ulic) oraz innych jednostek nazwanych, takich jak osoby czy organizacje. Możesz wykorzystać ten model do identyfikacji podstawowych elementów w danych teleadresowych.

In [1]:
import spacy

# Załaduj polski model językowy
nlp = spacy.load("pl_core_news_sm")

# Tekst do analizy
text = "Kraków, ul. Floriańska 10, 30-001, tel. 123-456-789"

# Przetwarzanie tekstu
doc = nlp(text)

# Wyodrębnianie encji nazwanych
for ent in doc.ents:
    print(f"Tekst: {ent.text}, Typ: {ent.label_}")

Tekst: Kraków, Typ: placeName
Tekst: ul., Typ: geogName
Tekst: Floriańska, Typ: geogName


## 2. Rozszerzenie modelu o nowe encje

Jeśli wbudowany model spaCy nie rozpoznaje specyficznych elementów (np. numerów telefonów, kodów pocztowych), możesz dodać własne reguły lub wytrenować model na własnych danych.

Dodanie reguł (matcher):

Możesz użyć mechanizmu **Matcher** lub **EntityRuler**, aby rozpoznawać specyficzne wzorce, takie jak numery telefonów czy kody pocztowe.



In [2]:
from spacy.matcher import Matcher

# Inicjalizacja matcher
matcher = Matcher(nlp.vocab)

# Definicja wzorców (np. numer telefonu, kod pocztowy)
phone_pattern = [{"SHAPE": "ddd"}, {"TEXT": "-"}, {"SHAPE": "ddd"}, {"TEXT": "-"}, {"SHAPE": "ddd"}]
postal_code_pattern = [{"SHAPE": "ddd"}, {"TEXT": "-"}, {"SHAPE": "ddd"}]

# Dodanie wzorców
matcher.add("PHONE_NUMBER", [phone_pattern])
matcher.add("POSTAL_CODE", [postal_code_pattern])

# Tekst do analizy
text = "Kraków, ul. Floriańska 10, 30-001, tel. 123-456-789"
doc = nlp(text)

# Dopasowanie wzorców
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    print(f"Znaleziono: {span.text}, Typ: {nlp.vocab.strings[match_id]}")

Znaleziono: 123-456, Typ: POSTAL_CODE
Znaleziono: 123-456-789, Typ: PHONE_NUMBER
Znaleziono: 456-789, Typ: POSTAL_CODE


## 3. Własne wytrenowanie modelu spaCy

Jeśli masz dużo danych teleadresowych, możesz wytrenować model **spaCy** na ich podstawie.

**Przygotowanie danych treningowych:**

Dane muszą być w formacie:

In [4]:
TRAIN_DATA = [
    ("Kraków, ul. Floriańska 10, 30-001", {"entities": [(0, 6, 'placeName'), (8, 25, 'streetName'), (27, 33, 'postalCode')]}),
    ("Warszawa, Aleje Jerozolimskie 5, 01-234", {"entities": [(0, 8, 'placeName'), (10, 31, 'streetName'), (33, 39, 'postalCode')]}),
]

#### Funkcja sprawdzająca wyrównanie encji danych

In [12]:
from spacy.training import offsets_to_biluo_tags
from spacy.tokens import Doc

# Załaduj model
nlp = spacy.blank("pl")

# Przykładowe dane
text = "Kraków, ul. Floriańska 10, 30-001"
entities = [(0, 6, 'placeName'), (8, 25, 'streetName'), (27, 33, 'postalCode')]

# Sprawdź wyrównanie encji
doc = nlp.make_doc(text)
biluo_tags = offsets_to_biluo_tags(doc, entities)
print(biluo_tags)

['U-placeName', 'O', 'B-streetName', 'I-streetName', 'I-streetName', 'L-streetName', 'O', 'B-postalCode', 'I-postalCode', 'L-postalCode']


#### Trenowanie modelu

In [13]:
import spacy
from spacy.training.example import Example
from spacy.tokens import DocBin

# Przygotowanie modelu
nlp = spacy.blank("pl")  # Pusty model dla języka polskiego
ner = nlp.add_pipe("ner")

# Dodanie etykiet
ner.add_label("placeName")
ner.add_label("streetName")
ner.add_label("postalCode")

# Tworzenie danych treningowych
examples = []
for text, annotations in TRAIN_DATA:
    doc = nlp.make_doc(text)
    examples.append(Example.from_dict(doc, annotations))

# Trenowanie modelu
optimizer = nlp.begin_training()
for epoch in range(80):
    losses = {}
    nlp.update(examples, losses=losses, drop=0.3)
    if epoch % 5 == 0:
      print(f"Epoch {epoch}, Losses: {losses}")

# Zapis modelu
nlp.to_disk("teleaddress_model")

Epoch 0, Losses: {'ner': 17.642855644226074}
Epoch 5, Losses: {'ner': 15.36121779680252}
Epoch 10, Losses: {'ner': 9.671059787273407}
Epoch 15, Losses: {'ner': 6.335512733552605}
Epoch 20, Losses: {'ner': 11.140499805100262}
Epoch 25, Losses: {'ner': 15.634618883254007}
Epoch 30, Losses: {'ner': 10.811726793646812}
Epoch 35, Losses: {'ner': 7.942607365548611}
Epoch 40, Losses: {'ner': 4.149004853796214}
Epoch 45, Losses: {'ner': 1.5916382318828255}
Epoch 50, Losses: {'ner': 0.21559912930388236}
Epoch 55, Losses: {'ner': 0.008842473442200571}
Epoch 60, Losses: {'ner': 6.785762597938394e-06}
Epoch 65, Losses: {'ner': 7.029231605045448e-06}
Epoch 70, Losses: {'ner': 2.815597292332452e-08}
Epoch 75, Losses: {'ner': 3.030730690203496e-06}


#### Użycie wytrenowanego modelu:


In [14]:
nlp = spacy.load("teleaddress_model")
doc = nlp("Kraków, ul. Floriańska 10, 30-001")
for ent in doc.ents:
    print(f"Tekst: {ent.text}, Typ: {ent.label_}")

Tekst: Kraków, Typ: placeName
Tekst: ul. Floriańska 10, Typ: streetName
Tekst: 30-001, Typ: postalCode


## 4. Integracja z innymi narzędziami

Możesz łączyć spaCy z innymi bibliotekami:

- **re**: Rozpoznawanie wzorców za pomocą wyrażeń regularnych.
- **rapidfuzz**: Dopasowanie danych do list miast lub ulic.
- **pandas**: Strukturalizacja i analiza danych wyodrębnionych z tekstu.

In [21]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("pl_core_news_sm")
matcher = Matcher(nlp.vocab)

# Definicja wzorców
phone_pattern = [{"SHAPE": "ddd"}, {"TEXT": "-"}, {"SHAPE": "ddd"}, {"TEXT": "-"}, {"SHAPE": "ddd"}]
postal_code_pattern = [{"SHAPE": "ddd"}, {"TEXT": "-"}, {"SHAPE": "ddd"}]

matcher.add("PHONE_NUMBER", [phone_pattern])
matcher.add("POSTAL_CODE", [postal_code_pattern])

# Tekst do rozpoznania
text = "Kraków, ul. Floriańska 10, 30-001, tel. 123-456-789"

doc = nlp(text)

results = {"placeName": None, "streetName": None, "postalCode": None, "phoneNumber": None}

# Rozpoznawanie encji
for ent in doc.ents:
    if ent.label_ == "placeName":
        results["placeName"] = ent.text
    elif ent.label_ == "streetName":
        results["streetName"] = ent.text

# Dopasowanie wzorców
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    if nlp.vocab.strings[match_id] == "PHONE_NUMBER":
        results["phoneNumber"] = span.text
    elif nlp.vocab.strings[match_id] == "POSTAL_CODE":
        results["postalCode"] = span.text

print(results)

{'placeName': 'Kraków', 'streetName': 'Floriańska 10', 'postalCode': '456-789', 'phoneNumber': '123-456-789'}
